Обработка для создания списка компаний с наличием паспорта

In [2]:
import os
import pandas as pd

In [2]:
input_folder = "passports"
output_folder = "passports/unique_passports"
os.makedirs(output_folder, exist_ok=True)

In [3]:
files = [f for f in os.listdir(input_folder) if f.startswith("companies_passport_") and f.endswith(".csv")]
all_unique_companies = pd.DataFrame(columns=["name"])

In [4]:
#  Создание списков уникальных компаний для каждой отрасли и общего списка уникальных компаний
for file in files:
    industry_name = file.replace("companies_passport_", "").replace(".csv", "")
    file_path = os.path.join(input_folder, file)
    df = pd.read_csv(file_path, header=None, usecols=[0], names=["name"])
    unique_companies = df.drop_duplicates(subset=["name"])
    output_file_path = os.path.join(output_folder, f"ucp_{industry_name}.csv")
    unique_companies.to_csv(output_file_path, index=False, header=False)
    all_unique_companies = pd.concat([all_unique_companies, unique_companies])
all_unique_companies = all_unique_companies.drop_duplicates(subset=["name"])
all_unique_companies.to_csv(os.path.join(output_folder, "all_ucp.csv"), index=False, header=False)

In [5]:
all_ucp_df = pd.read_csv(os.path.join(output_folder, "all_ucp.csv"), header=None, names=["name"])
matched_results_folder = "matched_results"
unique_mc_df = pd.read_csv(os.path.join(matched_results_folder, "unique_mc.csv"), header=None, names=["name"])
not_in_unique_mc = all_ucp_df[~all_ucp_df["name"].isin(unique_mc_df["name"])]
not_in_all_ucp = unique_mc_df[~unique_mc_df["name"].isin(all_ucp_df["name"])]
result = pd.concat([not_in_unique_mc, not_in_all_ucp])
result.to_csv(os.path.join(output_folder, "!all_ucp.csv"), index=False, header=False)

In [15]:
# Создание объединенного списка компаний с паспортами

input_folder = "passports/passports"
output_file = os.path.join(input_folder, "companies_passport.csv")

files = [f for f in os.listdir(input_folder) if f.startswith("companies_passport_") and f.endswith(".csv")]

combined_df = pd.DataFrame()

for file in files:
    file_path = os.path.join(input_folder, file)
    df = pd.read_csv(file_path, header=None)
    combined_df = pd.concat([combined_df, df])

combined_df = combined_df.drop_duplicates()

combined_df.to_csv(output_file, index=False, header=False)

In [ ]:
#  Добавление колонки с названием отрасли из отпарсенных паспортов

input_folder = "matched_results"
output_file = os.path.join(input_folder, "dataset.csv")

files = [f for f in os.listdir(input_folder) if f.startswith("matched_") and f.endswith(".csv")]

combined_df = pd.DataFrame()

for file in files:
    industry_name = file.replace("matched_", "").replace(".csv", "")
    file_path = os.path.join(input_folder, file)
    df = pd.read_csv(file_path, header=0)
    df['ind'] = industry_name
    combined_df = pd.concat([combined_df, df])

combined_df = combined_df.drop_duplicates()

combined_df.to_csv(output_file, index=False, header=True)

In [ ]:
# Удаление из основного датасета строк, где у компаний нет паспортов

all_ucp_df = pd.read_csv('matched_results/!all_ucp.csv', header=None, names=['Компания'])
dataset_path = 'matched_results/dataset.csv'
df = pd.read_csv(dataset_path)
filtered_df = df[~df['Компания'].isin(all_ucp_df['Компания'])]
filtered_df.to_csv('dataset.csv', index=False)

In [ ]:
# Объединение списков финал
df_p2 = pd.read_csv('df_p2.csv', header=None)
df = pd.read_csv('df.csv')
df_p2.columns = ['Компания', 'Город', 'Продукт', 'Технология', 'Год']
print(df_p2.head())
merged_df = pd.merge(df, df_p2[['Компания', 'Год', 'Продукт', 'Технология']], left_on=['Компания', 'year'], right_on=['Компания', 'Год'], how='left')
merged_df = merged_df.drop(columns=['Год'])
merged_df.to_csv('MainDF.csv', index=False)

                                Компания     Город  \
0                 Стройсервис (Кемерово)  Кемерово   
1                 Стройсервис (Кемерово)  Кемерово   
2  Атлас Майнинг (ранее Петропаловск ГК)    Москва   
3  Атлас Майнинг (ранее Петропаловск ГК)    Москва   
4  Атлас Майнинг (ранее Петропаловск ГК)    Москва   

                                      Продукт  \
0  VisionLabs Thermo (ранее VisionLabs Termo)   
1                             TrueConf Server   
2                                 Directum RX   
3                           Directum Projects   
4                           Datareon Platform   

                                          Технология   Год  
0  ИБ - Биометрическая идентификация,Системы виде...  2021  
1                           SaaS,Видеоконференцсвязь  2019  
2                                   BPM,HRM,SaaS,СЭД  2022  
3                       Системы управления проектами  2021  
4  Data Quality - Качество данных,MDM - Master Da...  2016  


C:\Users\Home\AppData\Local\Temp\ipykernel_25860\1933984628.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('df.csv')


In [4]:
industries_folder = "industries_df"
files = [f for f in os.listdir(industries_folder) if f.endswith(".csv")]

row_counts = {}

for file in files:
    file_path = os.path.join(industries_folder, file)
    df = pd.read_csv(file_path)
    row_counts[file] = len(df)

total_rows = sum(row_counts.values())
print(total_rows)

26171


In [6]:
main_df = pd.read_csv('MainDF.csv')

columns_to_remove = ['legal_form', 'ownership', 'date_registration', 'date_liquidation',
 'name_full', 'address', 'head_inn', 'date_liquidation', 'name_eng', 'registration_num',
   'firm_inn', 'legal_form', 'ownership', 'certified_product', 'name_short',
    'self_regulatory', 'okved_section', 'employees_full']  
main_df = main_df.drop(columns=columns_to_remove, errors='ignore')

main_df.to_csv('MainDF.csv', index=False)

# Подсчет количества уникальных компаний
unique_companies_count = main_df['Компания'].nunique()
print(f"Количество уникальных компаний: {unique_companies_count}")

Количество уникальных компаний: 2009


In [7]:
MainDF = pd.read_csv('MainDF.csv')
MainDF.head()

,Компания,Город,name,region.y,industry,claims_amount,proceedings_amount,size_cat,employees,income,...,ros,year,code_okved,firts_number_okved,okved_name,retained_earnings,short_term_liab,ind,Продукт,Технология
0,Археологические изыскания в строительстве (АИСТ),Москва,"АРХЕОЛОГИЧЕСКИЕ ИЗЫСКАНИЯ В СТРОИТЕЛЬСТВЕ, ООО",Москва,Расчистка территории строительной площадки,NaN,NaN,Малые предприятия,38.0,129727.0,...,0.2059,2017,43.12.1,43.0,Строительство,NaN,NaN,dobycha_poleznyh_iskopaemyh,NaN,NaN
1,Археологические изыскания в строительстве (АИСТ),Москва,"АРХЕОЛОГИЧЕСКИЕ ИЗЫСКАНИЯ В СТРОИТЕЛЬСТВЕ, ООО",Москва,Расчистка территории строительной площадки,NaN,NaN,Малые предприятия,41.0,141990.0,...,0.2001,2019,43.12.1,43.0,Строительство,24271000.0,25793000.0,dobycha_poleznyh_iskopaemyh,NaN,NaN
2,Археологические изыскания в строительстве (АИСТ),Москва,"АРХЕОЛОГИЧЕСКИЕ ИЗЫСКАНИЯ В СТРОИТЕЛЬСТВЕ, ООО",Москва,Расчистка территории строительной площадки,NaN,NaN,Малые предприятия,36.0,120776.0,...,0.2449,2018,43.12.1,43.0,Строительство,13510000.0,27874000.0,dobycha_poleznyh_iskopaemyh,NaN,NaN
3,Археологические изыскания в строительстве (АИСТ),Москва,"АРХЕОЛОГИЧЕСКИЕ ИЗЫСКАНИЯ В СТРОИТЕЛЬСТВЕ, ООО",Москва,Расчистка территории строительной площадки,NaN,NaN,Малые предприятия,51.0,258674.0,...,0.4590,2022,43.12.1,43.0,Строительство,27646000.0,34800000.0,dobycha_poleznyh_iskopaemyh,NaN,NaN
4,Археологические изыскания в строительстве (АИСТ),Москва,"АРХЕОЛОГИЧЕСКИЕ ИЗЫСКАНИЯ В СТРОИТЕЛЬСТВЕ, ООО",Москва,Расчистка территории строительной площадки,NaN,NaN,Малые предприятия,NaN,NaN,...,0.0882,2015,43.12.1,43.0,Строительство,NaN,NaN,dobycha_poleznyh_iskopaemyh,NaN,NaN
